In [2]:
import pandas as pd 
import numpy as np

The data was wrangled after importing the data into data frames. A data frame, which contains the count of each user id in terms of visits was created. That data frame was them merge with the user information dataframe. Since there is a column which contains data regarding weather or not the user was invited, etc., the column which contains the data as to which other user invited was deleted. Along with columns such as emails, names, etc, because the name and emails of user does not have relavence. I then deleted the dateimte columns for simplicity. 

I then began with making a linear regression to see what factors best serve as predictors. I did not want to, at first, do a classification model because it did not seem to make sense to classify those who visited over 100 times in the same category as those that visited 3 times. But after seeing the score of the raw linear model of an R squared being 0.001, I decided to try a classification model. I made a column that is known as user adoption column which has values of true and false, true for those who have visited 3 or more times. I made that into a dummy variable and built a logitistic regression model to classify those that are adopted. After construction, I saw that the f1 score for those that were adopted was 0. The variables are not sufficient in classifying correctly. There are no regressors, only categorical and dummies were used. I would suggest, to get more information about each of the users, such as their age and other information relavent to the product. 

In [3]:
df = pd.read_csv('takehome_users.csv', sep=',', encoding='latin-1')
df1 = pd.read_csv('takehome_user_engagement.csv')

In [4]:
dfa = df1.groupby('user_id').visited.count()

In [7]:
dfa = pd.DataFrame(dfa)

In [8]:
dfb = pd.merge(left=df, right=dfa, left_on='object_id', right_on='user_id')

In [9]:
dfb.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,visited
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,1
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,14
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,1
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,1
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,1


In [10]:
del dfb['invited_by_user_id']
del dfb['email']
del dfb['last_session_creation_time']
del dfb['name']
del dfb['creation_time']
dfb = dfb.set_index('object_id')

In [11]:
dfb.head()

,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,visited
object_id,,,,,
1,GUEST_INVITE,1,0,11,1
2,ORG_INVITE,0,0,1,14
3,ORG_INVITE,0,0,94,1
4,GUEST_INVITE,0,0,1,1
5,GUEST_INVITE,0,0,193,1


In [12]:
dfb.enabled_for_marketing_drip = dfb.enabled_for_marketing_drip.astype('category')
dfb.opted_in_to_mailing_list = dfb.opted_in_to_mailing_list.astype('category')
dfb.creation_source = dfb.creation_source.astype('category')

In [13]:
dfb = pd.get_dummies(dfb, drop_first=True)

In [16]:
dfb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8823 entries, 1 to 12000
Data columns (total 8 columns):
org_id                                8823 non-null int64
visited                               8823 non-null int64
creation_source_ORG_INVITE            8823 non-null uint8
creation_source_PERSONAL_PROJECTS     8823 non-null uint8
creation_source_SIGNUP                8823 non-null uint8
creation_source_SIGNUP_GOOGLE_AUTH    8823 non-null uint8
opted_in_to_mailing_list_1            8823 non-null uint8
enabled_for_marketing_drip_1          8823 non-null uint8
dtypes: int64(2), uint8(6)
memory usage: 258.5 KB


In [17]:
X = dfb.drop('visited', axis=1)
y = dfb['visited']

In [40]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=42)
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
reg.score(X_test,y_test)

0.0019883352987417613

In [21]:
dfb['adopted_user'] = dfb.visited>2

In [22]:
dfb.adopted_user = dfb.adopted_user.astype('category')

In [24]:
dfb = pd.get_dummies(dfb, drop_first=True)

In [35]:
dfb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8823 entries, 1 to 12000
Data columns (total 9 columns):
org_id                                8823 non-null int64
visited                               8823 non-null int64
creation_source_ORG_INVITE            8823 non-null uint8
creation_source_PERSONAL_PROJECTS     8823 non-null uint8
creation_source_SIGNUP                8823 non-null uint8
creation_source_SIGNUP_GOOGLE_AUTH    8823 non-null uint8
opted_in_to_mailing_list_1            8823 non-null uint8
enabled_for_marketing_drip_1          8823 non-null uint8
adopted_user_True                     8823 non-null uint8
dtypes: int64(2), uint8(7)
memory usage: 267.1 KB


In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
X2 = dfb.drop(columns=['visited','adopted_user_True'])

In [28]:
y2 = dfb.adopted_user_True

In [38]:
logreg = LogisticRegression()
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.25, random_state=42)
logreg.fit(X2_train, y2_train)
y2_pred = logreg.predict(X2_test)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [39]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y2_test,y2_pred))
print(classification_report(y2_test, y2_pred))

[[1618    0]
 [ 588    0]]
              precision    recall  f1-score   support

           0       0.73      1.00      0.85      1618
           1       0.00      0.00      0.00       588

   micro avg       0.73      0.73      0.73      2206
   macro avg       0.37      0.50      0.42      2206
weighted avg       0.54      0.73      0.62      2206



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
